# +AUC +sen +spe 转test3计算均值
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn import datasets
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
import time
from pylab import rcParams
import matplotlib.cm as cm
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder  # 编码转换
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier  # 随机森林
from sklearn.svm import SVC, LinearSVC  # 支持向量机
from sklearn.linear_model import LogisticRegression  # 逻辑回归
from sklearn.neighbors import KNeighborsClassifier  # KNN算法
from sklearn.naive_bayes import GaussianNB  # 朴素贝叶斯
from sklearn.tree import DecisionTreeClassifier  # 决策树分类器
# from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score,accuracy_score,roc_auc_score
from sklearn.metrics import confusion_matrix
#import sys 
#print(sys.path)
#print("当前路径 -> %s" %os.getcwd())

import warnings
warnings.filterwarnings("ignore")

class Nstr:
    def __init__(self,arg):
        self.x=arg
    def __sub__(self,other):
        c=self.x.replace(other.x,"")
        return c



filename= os.listdir(r'C:\\Users\Xia HAN\\Downloads\\jupyter_code_data_WEI_kunni\\code_20221124\\color4')##
                           
#print(filename) ## filename为该表格的所有sheet，如'sheet1','sheet2','sheet3'

for i in range(len(filename)):
    print("正在处理第"+str(i)+"文件，")
#for i in range(0,1):    #########此行为测试时用
    totaldata = pd.read_excel(filename[i], sheet_name=4)
    totaldata.dropna(inplace=True)
    for j in range(3,12):###控制特征向量的长度
    #for j in range(3,4):  #########此行为测试时用
        #print(filename[i])##########此行为测试用
        print("此时特征向量长度为"+str(j-2)+"...")
        brain = totaldata.iloc[:, 2:j]  ##############################################更改特征向量长度,从1-9
        #print(brain)
        scaler = StandardScaler(copy=False)
        scaler.fit_transform(brain)
        
        brain = scaler.fit_transform(brain)
        k= 100                                 ##########################################100  修改计算次数
        X=brain
        
        y=totaldata["group"].values
        sum_result = 0
        list_result=[]
        list_result_time=[]
        Classifiers = [
            ["Random Forest", RandomForestClassifier()],
            ["Support Vector Machine", SVC()],
            ["LogisticRegression",LogisticRegression()],
            ["KNN",KNeighborsClassifier(n_neighbors=5)],    ########### K=5
            # ["Naive Bayes",GaussianNB()],
            ["Decision Tree",DecisionTreeClassifier()],
            # ["AdaBoostClassifier", AdaBoostClassifier()],
            # ["GradientBoostingClassifier", GradientBoostingClassifier()],
            # ["XGB", XGBClassifier()],
            ["CatBoost", CatBoostClassifier(logging_level='Silent')]
        ]
        sss = StratifiedShuffleSplit(n_splits=k, test_size=0.3, random_state=None)# n_splits：是将训练数据分成train/test对的组数
                                                                                   # train：test=7：3
        for train_index, test_index in sss.split(X, y):
            #print("train:", train_index, "test:", test_index)
            X_train,X_test=X[train_index], X[test_index]
            y_train,y_test=y[train_index], y[test_index]
            
            Classify_result = []
            names = []
            prediction = []
            times=[]
            
            for name, classifier in Classifiers:   ##此时分类器类别个数为6
                #print(name)
                #print()
                start = time.time()
                classifier = classifier
                classifier.fit(X_train, y_train)
                y_pred = classifier.predict(X_test)
                acc_score = accuracy_score(y_test, y_pred)
                
                class_eva = pd.DataFrame([acc_score])
                Classify_result.append(class_eva)
                
                name = pd.Series(name)
                names.append(name)
                
                end = time.time()
                exe_time = str(round((end-start),6))
                
                exe_time = pd.DataFrame([exe_time])
                times.append(exe_time)
                
                #print(exe_time)
                
            names = pd.DataFrame(names)  ##这里的names是分类器的名称
            ##print(names)
            names = names[0].tolist()
            ##print(names[0])
            result = pd.concat(Classify_result, axis=1)#axis=1为列合并
            result_time=pd.concat(times,axis=1)
            ##print(result) 
            result.columns = names
            result_time.columns = names
            result.index = ["acc_score"]
            result_time.index = ["exe_time"]
            list_result.append(result)
            list_result_time.append(result_time)

            #sum_result += result
        avg_result = pd.concat(list_result, axis=0) ##axis=0为基于行合并
        #print(avg_result)
        avg_result_time = pd.concat(list_result_time, axis=0)
        
        #print(exe_time)
        
        fileName_xlsx = Nstr(str(filename[i]))
        fileName_suffix = Nstr(".xlsx")
        new_fileName = fileName_xlsx - fileName_suffix
        new_fileName = new_fileName+"_"+str(j-2)+"_"
        avg_result.to_csv((new_fileName+"vectorLongeur"+".csv"))  ###写文件这里
        avg_result_time.to_csv((new_fileName+"exeTime"+".csv"))
        
        ##time.sleep(1)